In [38]:
import tensorflow as tf
import sys
print(sys.version)
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt

3.5.2 |Continuum Analytics, Inc.| (default, Jul  2 2016, 17:53:06) 
[GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]


In [39]:
class MF_RS():
    def __init__(self, numUsers, numSongs, embedding_dim, reg_lambda = 0.01):
        
        #hyper parameters
        self.batch_size = np.min([256, numUsers, numSongs]);
        self.numUsers = numUsers
        self.numSongs = numSongs
        self.epochs = 50
        self.reg_lambda = reg_lambda
        
        #embedding matricies for users and songs
        self.userMat = tf.Variable(tf.random_normal([numUsers, embedding_dim]))
        self.songMat = tf.Variable(tf.random_normal([numSongs, embedding_dim]))
        self.userBias = tf.Variable(tf.random_normal([numUsers]))
        self.songBias = tf.Variable(tf.random_normal([numSongs]))
        self.overallBias = tf.Variable(tf.random_normal([1]))
        
        #input tensors for songs, usres, ratings
        self.users = tf.placeholder(tf.int32, shape =(self.batch_size))
        self.songs = tf.placeholder(tf.int32, shape =(self.batch_size))
        self.rating = tf.placeholder(tf.float32, shape = (self.batch_size))
        
        #map each user/song to its feature vector
        self.U = tf.nn.embedding_lookup(self.userMat, self.users)
        self.W = tf.nn.embedding_lookup(self.songMat, self.songs)
        #map each user/song bias to its bias vector
        self.U_bias = tf.nn.embedding_lookup(self.userBias, self.users)
        self.W_bias = tf.nn.embedding_lookup(self.songBias, self.songs)
        
        #predicted rating is dot product of user and song
        bias = self.U_bias+self.W_bias+self.overallBias
        pq = tf.reduce_sum(tf.mul(self.U, self.W), 1)
        self.yhat = pq + bias
        
        self.reg = self.reg_lambda * ( tf.reduce_sum((tf.square(self.U) + tf.square(self.W))) + 
                                 tf.reduce_sum(tf.square(self.U_bias) + tf.square(self.W_bias)))
        self.error = tf.reduce_mean(tf.nn.l2_loss(self.yhat - self.rating))
        self.cost = (self.error + self.reg)/1e4
        self.optimizer = tf.train.AdamOptimizer(learning_rate = .01).minimize(self.cost)
        
        self.session = tf.Session()
        self.session.run(tf.initialize_all_variables())   
        
        
    def train(self, users, songs, ratings):
        
        for i in range(self.epochs):
            
            avg_cost = 0
            perm = np.random.permutation(len(ratings))
            num_batches = len(ratings) // self.batch_size
            
            for b_idx in range(num_batches):
                
                batch = perm[self.batch_size * b_idx:self.batch_size * (b_idx + 1)]
                users_batch = users[batch]
                songs_batch = songs[batch]
                ratings_batch = ratings[batch]
                                
                avg_cost += self.session.run([self.cost, self.optimizer],
                          {self.users:users_batch, self.songs:songs_batch, self.rating:ratings_batch})[0]
                
            print(avg_cost/num_batches)
    def test(self, users, songs):
        yhat = np.zeros(len(users))
        num_batches = len(users) // self.batch_size
        for b_idx in range(num_batches):
            batch = range(self.batch_size * b_idx,self.batch_size * (b_idx + 1))
            users_batch = users[batch]
            songs_batch = songs[batch]
            yhat[batch] = self.session.run([self.yhat],
                      {self.users:users_batch, self.songs:songs_batch})[0]
        batch = range(-self.batch_size,0)
        users_batch = users[batch]
        songs_batch = songs[batch]
        yhat[batch] = self.session.run([self.yhat],
                      {self.users:users_batch, self.songs:songs_batch})[0]
        return yhat
    def evaluate(self, users, songs, ratings):
        yhat = self.test(users, songs)
        return np.mean((yhat - ratings)**2)

In [40]:
a = np.array([1, 2, 3, 4, 5])
b = np.array([1, 2, 3, 4, 5])
c = np.array([4, 3, 2, 5, 1])
#unique users / songs
uni_a = np.unique(a)
uni_b = np.unique(b)

#dict mapping the id to an index
a_map = dict(zip(uni_a,range(len(uni_a))))
b_map = dict(zip(uni_b,range(len(uni_b))))

user_idx =  np.array([ a_map[user] for user in a])
song_idx =  np.array([ b_map[song] for song in b])
model = MF_RS(len(uni_a), len(uni_b), 7)
np.random.seed(1)
model.train(user_idx, song_idx, c)


0.00192653317936
0.00177187134977
0.00162790284958
0.0014943171991
0.00137014803477
0.00125386985019
0.00114414468408
0.00104041059967
0.000942704384215
0.000851274409797
0.000766346638557
0.000688024098054
0.000616248289589
0.000550792203285
0.000491286860779
0.000437282171333
0.000388326821849
0.000344035302987
0.000304115877952
0.000268357835012
0.000236596461036
0.000208671568544
0.000184396412806
0.000163539414643
0.000145824480569
0.000130947708385
0.000118602511066
0.000108503030788
0.000100396726339
9.40637328313e-05
8.93055694178e-05
8.59302526806e-05
8.37400584714e-05
8.2526152255e-05
8.20721470518e-05
8.21648354759e-05
8.26087052701e-05
8.32383520901e-05
8.39250860736e-05
8.45766626298e-05
8.51308213896e-05
8.55469261296e-05
8.57977502164e-05
8.58650601003e-05
8.57381673995e-05
8.54158424772e-05
8.49084826768e-05
8.42388326419e-05
8.34402235341e-05
8.25522802188e-05


In [41]:
movieratings = pd.read_csv('ratings.csv')

In [42]:
#movieratings.describe()

In [43]:
def getDfSummary(input_data):
    output_data = input_data.describe(include = 'all').T
    var = pd.DataFrame(data = {'nanvals': pd.Series(), 'number_distinct': pd.Series()})
    for i in range(len(input_data.columns)):
        nanvals = input_data.ix[:,i].isnull().sum()
        number_distinct = len(input_data.ix[:,i].value_counts())
        var = var.append(pd.DataFrame([[nanvals, number_distinct]], columns = ['nanvals', 'number_distinct']))
    var.index = output_data.index.values
    output_data['nanvals'] = var['nanvals']
    output_data['number_distinct'] = var['number_distinct']
    return output_data
output_data = getDfSummary(movieratings)

In [44]:
users = movieratings.ix[:,0].values
songs = movieratings.ix[:,1].values
ratings = movieratings.ix[:,2].values

#unique users / songs
uni_users = movieratings['userId'].unique()
uni_songs = movieratings['movieId'].unique()

#dict mapping the id to an index
user_map = dict(zip(uni_users,range(len(uni_users))))
song_map = dict(zip(uni_songs,range(len(uni_songs))))

user_idx =  np.array([ user_map[user] for user in users])
song_idx =  np.array([ song_map[song] for song in songs])

print(len(uni_users),len(uni_songs))

perm = np.random.permutation(len(users))
trn_idx = perm[:(len(users)*2)//3]
val_idx = perm[(len(users)*2)//3:]
user_idx_trn, song_idx_trn, ratings_trn = user_idx[trn_idx], song_idx[trn_idx], ratings[trn_idx]
user_idx_val, song_idx_val, ratings_val = user_idx[val_idx], song_idx[val_idx], ratings[val_idx]

671 9066


In [45]:
songmodel = MF_RS(len (uni_users), len(uni_songs), 11)

In [46]:
songmodel.evaluate(user_idx_val, song_idx_val, ratings_val)

41.87104695957666

In [47]:
songmodel.train(user_idx_trn, song_idx_trn, ratings_trn)

0.27106878981
0.0688052115389
0.0348479230243
0.0229122875115
0.0172888871497
0.0142257605285
0.0123542843816
0.0111158553905
0.010255866247
0.0095995343738
0.00907895249279
0.00863624070592
0.00824814682217
0.00789690037759
0.00756349391662
0.00725510769273
0.00695274841363
0.00668187545648
0.0064159199882
0.00619064112946
0.00598286432393
0.00580430886207
0.00564636590783
0.00551073051219
0.00538029499943
0.00529170126403
0.00519569661612
0.00510854806321
0.00505068278871
0.00498612901787
0.00493253018403
0.00487570496312
0.0048333545099
0.0047929958315
0.0047574156555
0.00471707085649
0.00468999624987
0.00465341930618
0.00462047924073
0.00460539436989
0.00458932031985
0.00455232743317
0.00453562270492
0.00451947565083
0.00450428698857
0.00448990591999
0.00447327825241
0.00445131833773
0.00444595325613
0.00443008510718


In [48]:
songmodel.evaluate(user_idx_val, song_idx_val, ratings_val)

1.2080486996728497